In [3]:
# Imports
import os
from django.apps import apps as django_apps

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [8]:
from edc_appointment.models import Appointment
from edc_visit_schedule.site_visit_schedules import site_visit_schedules

idx = 'B142-040990562-6'
child_idx = 'B142-040990562-6-10'
enrol_onsch = OnScheduleCohortAEnrollment.objects.get(subject_identifier=idx,
                                                      child_subject_identifier=child_idx)
visit = MaternalVisit.objects.get(subject_identifier=enrol_onsch.subject_identifier,
                                  schedule_name=enrol_onsch.schedule_name)
sec_onsch = OnScheduleCohortBSecQuart.objects.get(subject_identifier=idx)
sec_ssh = SubjectScheduleHistory.objects.get(
    subject_identifier=sec_onsch.subject_identifier,
    schedule_name=sec_onsch.schedule_name)
sec_onsch.delete()
sec_ssh.delete()

_, schedule = site_visit_schedules.get_by_onschedule_model_schedule_name(
    onschedule_model=sec_ssh.onschedule_model, name=sec_ssh.schedule_name)
schedule.put_on_schedule(
    subject_identifier=sec_onsch.subject_identifier,
    onschedule_datetime=visit.report_datetime,
    schedule_name=sec_onsch.schedule_name,
    base_appt_datetime=visit.report_datetime)
sec_onsch = OnScheduleCohortBSecQuart.objects.get(subject_identifier=idx)
sec_onsch.child_subject_identifier = child_idx
sec_onsch.save()


In [9]:
visit_code = 2007
visit_codes = []
while visit_code >= 2001:
    visit_code -= 1
    visit_codes.append(f'{visit_code}M')

Appointment.objects.filter(subject_identifier=idx,
                           visit_code__in=visit_codes,
                           schedule_name=sec_onsch.schedule_name).delete()


(6, {'edc_appointment.Appointment': 6})

In [11]:
visit_code = 2007
while True:
    try:
        visit = MaternalVisit.objects.get(subject_identifier=idx, visit_code=f'{visit_code}M')
    except MaternalVisit.DoesNotExist:
        break
    else:
        appt = visit.appointment
        new_appt = Appointment.objects.filter(subject_identifier=idx,
                                              visit_code=f'{visit_code}M',
                                              schedule_name=sec_onsch.schedule_name)
        new_appt.update(appt_datetime=appt.appt_datetime,
                        appt_type=appt.appt_type,
                        appt_status=appt.appt_status,
                        appt_reason=appt.appt_reason,
                        comment=appt.comment)
        visit.appointment = new_appt[0]
        visit.schedule_name = new_appt[0].schedule_name
        visit.visit_schedule_name = new_appt[0].visit_schedule_name
        visit.save()
        appt.appt_status = 'new'
        appt.save()
        visit_code += 1

In [ ]:
a_quart_onsch = OnScheduleCohortAQuarterly.objects.get(subject_identifier=idx,
                                                      child_subject_identifier=child_idx)
quart_ssh = SubjectScheduleHistory.objects.get(subject_identifier=a_quart_onsch.subject_identifier,
                                               schedule_name=a_quart_onsch.schedule_name)
_, schedule = site_visit_schedules.get_by_onschedule_model_schedule_name(
    onschedule_model=quart_ssh.onschedule_model, name=quart_ssh.schedule_name)
schedule.take_off_schedule(subject_identifier=idx, schedule_name=a_quart_onsch.schedule_name)